In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.contrib import rnn
import time
import io
import math
import os
from io import StringIO
import pickle
sample_fq = 100
filename = 'timeserie_all_n_num.csv'

start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"
    
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean_diff = tf.reduce_mean(var, axis = 0)    
    
    mean = tf.reduce_mean(mean_diff)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))        
    tf.summary.scalar('stddev', stddev)
print("done")



done


In [2]:
def enc(y,num = 3):
    y_ = np.zeros((y.shape[0],3))
    for i in range(y.shape[0]):
        y_[i,int(y[i])] = 1
    #print(np.sum(np.sum(y_)),np.sum(y))
    return(y_)
    
    
def reader(filename,ind,batchsize,lenght):
    done = False
    str_ = ''
    with open(filename) as f:
        
        str_ += next(f)
        i = 1
        try:
            while done is False:
                i +=1

                if i < ind:
                    _ = next(f)
                elif i <= ind + batchsize:
                    str_ += next(f)
                else:
                    #print("no error during reading file i:",i," ind:",ind," batchsize:",batchsize," length:",lenght)
                    done = True           
        except:
            print("Error during reading file i:",i," ind:",ind," batchsize:",batchsize," length:",lenght)
            return reader(filename,int(np.floor(np.random.rand(1)*(lenght-batchsize-1))),batchsize,lenght)
        
        return StringIO(str_)
class fog_data:
    def __init__(self,filename,window_length = 250, batch_size = 32, split = [0.7,0.15,0.15]):
        
        self.agl = range(3,9)[:]
        print(self.agl)
        self.window_length = window_length
        self.batch_size = batch_size
        self.filename = filename
        self.filename_training = filename[:-4] + "_training.csv"
        self.filename_val = filename[:-4] + "_val.csv"        
        self.filename_test = filename[:-4] + "_test.csv"
        self.filename_dict = {}
        self.len_ = {}
        self.filename_dict["training"] = self.filename_training
        self.split = [sum(split[0:i]) for i in range(1,len(split))]
        if not os.path.isfile(self.filename_test):
            self.split_dataset()
        else:
            self.len_["training"] = 9392263
        self.filename_gan = filename[:-4] + '_gan.csv'
        if not os.path.isfile(self.filename_gan):
            self.makegan()
        else:
            self.len_["gan"] = 1159500
            self.filename_dict["gan"] = self.filename_gan
            
            
        self.len_["training_gan"] = 64000
        self.filename_training_gan = "train_gan.csv"
        self.filename_dict["training_gan"] = self.filename_training_gan
        
    def makegan(self):        

        df = pd.read_csv(self.filename)
        print("full data shape",df.shape,np.sum(df.FOGnFOG.values))
        
        gandata = []
        i = 50
        print(df.head(),df.iloc[0:10,8])
        while i <= (df.shape[0]-200):
            if df.FOGnFOG[i] == 1:
                
                gandata.append(df.iloc[i-50:i+200,:].values)
                i+=200
                if i% 100 == 0:
                    print("just added an extra part to the file l:",len(gandata)," i:",i)
            i += 1
                
        print(len(gandata))
        np_df = np.array(gandata).reshape((-1,10))
        print(np_df.shape,df.shape,np_df.shape[0]/df.shape[0])
        df = pd.DataFrame(data=np_df,columns=df.columns)
        self.len_["gan"] = df.shape[0]
        self.filename_dict["gan"] = self.filename_gan
        df.to_csv(self.filename_gan)
             
    def next_batch(self,type_of_data = "training", batchsize = 32*250):
        
        size = self.len_[type_of_data]
        filen = self.filename_dict[type_of_data]
        if type_of_data == "training":
            divider = 1
        elif type_of_data == "gan":
            divider = 250
        elif type_of_data == "training_gan":
            divider = 250
            if np.random.rand() < .5:
                type_of_data == "training"
                #print("going back to normal")
            #else:
                #print("keep beeing weird")
        
        ind = int(np.floor(np.random.rand(1)*(size-batchsize)/divider))
        df = pd.read_csv(reader(filen,ind*divider,batchsize,size))
        
        return self.return_data(df)
    
    def val(self):
        df = pd.read_csv(self.filename_val)
        return self.return_data(df,train = True)
    def test(self):
        df = pd.read_csv(self.filename_test)
        return self.return_data(df,train = True)
    

    def split_dataset(self):
        
        filenames = [self.filename_training,self.filename_val,self.filename_test]
        with open(self.filename,'r') as f:
            self.length = sum(1 for _ in f)
        with open(self.filename,'r') as f:
            self.split.append(self.length+1)
            self.split_i = np.floor([self.length * s for s in self.split])
            print(self.split_i)
            s = 0
            i = 0
            file = open(filenames[s],'w')
            for line in f:
                if i >= self.split_i[s]:
                    file.close()
                    s += 1
                    file = open(filenames[s],'w')
                    file.write(head)
                    print("switch to ",filenames[s])
                
                if i ==0:
                    head = line
                                        
                file.write(line)
                i +=1
                #print(i)
                
            file.close()
            print("splitted the data in different files according to ", [0, self.split_i, self.length])
            self.len_ = {}
            13417517
            self.len_["training"] = self.split_i[0]
            self.len_["val"], self.len_["test"] =  self.split_i[1] - self.split_i[0], self.length - self.split_i[1]
            
    def return_data(self,df,train = False): # include self.window_size
        acc_gyr = df.iloc[:,self.agl].values
        acc_gyr = (acc_gyr-np.mean(acc_gyr))/np.std(acc_gyr)
        num_windows = int(np.floor(np.divide(acc_gyr.shape[0],(self.window_length))))
        if acc_gyr.shape[0] - num_windows*self.window_length >1:
            print("through away something!!",acc_gyr.shape[0] - num_windows*self.window_length,acc_gyr.shape[0], num_windows*self.window_length)
            print(num_windows)
        a_g  = acc_gyr[:num_windows*self.window_length,:].reshape(num_windows,self.window_length,6)
        
        fog = df.FOGnFOG[:num_windows*self.window_length].values
        fog_reshape = fog.reshape(num_windows,self.window_length,1)
        fognfog = np.mean(fog_reshape,axis=1)
        if np.random.rand() < 0.95 and train:
            inx = np.random.randint(0,high=a_g.shape[0])
            a_g[inx,:,:] = (np.random.rand(a_g[inx,:,:].shape[0],a_g[inx,:,:].shape[1])*2-1)*a_g[inx,:,:]
            fognfog[inx,:] = 2
        
        
        return a_g, np.round(fognfog[:,0]), fognfog
    

fog = fog_data(filename)
acc, fog_, fogi = fog.next_batch()
a, f, fi = fog.next_batch("gan")
for _ in range(10):
    a, f, fi = fog.next_batch("training_gan")
print(acc.shape, fog_.shape," ee ", fogi.shape ,a.shape,f.shape,fi.shape)

range(3, 9)
through away something!! 249 7999 7750
31
(32, 250, 6) (32,)  ee  (32, 1) (32, 250, 6) (32,) (32, 1)


In [3]:
mb_size = 32
X_dim = 250* 6
h_dim = 512

tf.reset_default_graph()

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None,X_dim])
Y = tf.placeholder(tf.int32, shape = [None])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 3]))
D_b2 = tf.Variable(tf.zeros(shape=[3]))

def d(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    out = tf.matmul(D_h1, D_W2) + D_b2
    return tf.nn.softmax(out)

X_batch = tf.contrib.layers.batch_norm(X)

end = d(X_batch)
"""weighted_cross_entropy_with_logits(
    targets,
    logits,
    pos_weight,
    name=None
)"""
cross_entropy = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(labels = Y, logits = end))
tf.summary.scalar('mini_batch_cross_entropy', cross_entropy)
optimize = tf.train.AdamOptimizer(learning_rate = 1e-4).minimize(cross_entropy)


y_pred = tf.argmax(end,1)

merged = tf.summary.merge_all()

model_name = "FOG_512_2sd_try"
train_writer = tf.summary.FileWriter("tensorboard_FOG" + '/train/model_' + model_name)
val_writer = tf.summary.FileWriter("tensorboard_FOG" + '/val/model_' + model_name)
saver = tf.train.Saver( tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

print("start training")
start_time =time.time()
# Initializing the variables
b_size = 32
init = tf.global_variables_initializer()
i = 0
# Launch the graph
with tf.Session() as session:
    session.run(init)
    start_training_time = time.time()
    while i < 5000:
        i+=1
        X_mb, input_y, _ = fog.next_batch(batchsize = b_size*250)
        input_x = X_mb.reshape((-1,250*6))
        _, _summary = session.run([optimize, merged], feed_dict={X: input_x, Y: input_y})
        train_writer.add_summary(_summary, i * b_size )
    
        if i%250 is 0 and i is not 0: 
            print("current run time: ",time.time()-start_time,i)
            X_mb, input_y, _ = fog.val()       
            input_x = X_mb.reshape((-1,250*6))
            _, _summary = session.run([cross_entropy, merged ], feed_dict={X: input_x, Y: input_y})
            val_writer.add_summary(_summary, i * b_size )
    X_mb, input_y, _ = fog.val()       
    input_x = X_mb.reshape((-1,250*6))
    y_test = session.run([y_pred ], feed_dict={X: input_x, Y: input_y})
    pickle.dump([y_test,input_y],open('yval_normal.pickle','wb'))
    
    X_mb, input_y, _ = fog.test()       
    input_x = X_mb.reshape((-1,250*6))
    y_test = session.run([y_pred ], feed_dict={X: input_x, Y: input_y})
    pickle.dump([y_test,input_y],open('ytest_normal.pickle','wb'))


start training
current run time:  248.0745575428009 250
through away something!! 128 2012628 2012500
8050
current run time:  517.0489251613617 500
through away something!! 128 2012628 2012500
8050
current run time:  801.9047815799713 750
through away something!! 128 2012628 2012500
8050
current run time:  1086.5213084220886 1000
through away something!! 128 2012628 2012500
8050
current run time:  1362.0577828884125 1250
through away something!! 128 2012628 2012500
8050
current run time:  1649.4941756725311 1500
through away something!! 128 2012628 2012500
8050
current run time:  1913.9019417762756 1750
through away something!! 128 2012628 2012500
8050
current run time:  2182.68434381485 2000
through away something!! 128 2012628 2012500
8050
current run time:  2454.952432155609 2250
through away something!! 128 2012628 2012500
8050
current run time:  2739.6069507598877 2500
through away something!! 128 2012628 2012500
8050
current run time:  3010.420750141144 2750
through away something

In [4]:
mb_size = 32
X_dim = 250* 6
h_dim = 512

tf.reset_default_graph()

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None,X_dim])
Y = tf.placeholder(tf.int32, shape = [None])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 3]))
D_b2 = tf.Variable(tf.zeros(shape=[3]))

def d(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    out = tf.matmul(D_h1, D_W2) + D_b2
    return tf.nn.softmax(out)

X_batch = tf.contrib.layers.batch_norm(X)

end = d(X_batch)
"""weighted_cross_entropy_with_logits(
    targets,
    logits,
    pos_weight,
    name=None
)"""
cross_entropy = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(labels = Y, logits = end))
tf.summary.scalar('mini_batch_cross_entropy', cross_entropy)
optimize = tf.train.AdamOptimizer(learning_rate = 1e-4).minimize(cross_entropy)


y_pred = tf.argmax(end,1)

merged = tf.summary.merge_all()

model_name = "FOG_512_gan"
train_writer = tf.summary.FileWriter("tensorboard_FOG" + '/train/model_1080_' + model_name)
val_writer = tf.summary.FileWriter("tensorboard_FOG" + '/val/model_1080_' + model_name)
saver = tf.train.Saver( tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

print("start training")
start_time =time.time()
# Initializing the variables
b_size = 32
init = tf.global_variables_initializer()
i = 0
# Launch the graph
with tf.Session() as session:
    session.run(init)
    start_training_time = time.time()
    while i < 5000:
        i+=1
        X_mb, input_y, _ = fog.next_batch(type_of_data = "gan_training",batchsize = b_size*250)
        input_x = X_mb.reshape((-1,250*6))
        _, _summary = session.run([optimize, merged], feed_dict={X: input_x, Y: input_y})
        train_writer.add_summary(_summary, i * b_size )
    
        if i%250 is 0 and i is not 0: 
            print("current run time: ",time.time()-start_time,i)
            X_mb, input_y, _ = fog.val()       
            input_x = X_mb.reshape((-1,250*6))
            _, _summary = session.run([cross_entropy, merged ], feed_dict={X: input_x, Y: input_y})
            val_writer.add_summary(_summary, i * b_size )
    
    X_mb, input_y, _ = fog.val()       
    input_x = X_mb.reshape((-1,250*6))
    y_test = session.run([y_pred ], feed_dict={X: input_x, Y: input_y})
    pickle.dump([y_test,input_y],open('yval_gan.pickle','wb'))        
            
    X_mb, input_y, _ = fog.test()       
    input_x = X_mb.reshape((-1,250*6))
    y_test = session.run([y_pred ], feed_dict={X: input_x, Y: input_y})
    pickle.dump([y_test,input_y],open('ytest_gan.pickle','wb'))


start training


KeyError: 'gan_training'